In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tracker
import stats
import metrics
import visualizer
import utils

import importlib

In [ ]:
videofile = "SchmidtDataset/video_P001.mp4"

# Visualize the input
cap = cv2.VideoCapture(videofile)

# Get the first frame
ret, frame = cap.read()

plt.imshow(frame)

# Run Stats

In [ ]:
# prediction = "SchmidtDataset/video_P001_predicted.csv"
# groundtruth = "SchmidtDataset/video_P001_corrected.csv"
# subopt = "SchmidtDataset/video_P001_predicted_subopt.csv"
# videofile = "SchmidtDataset/video_P001.mp4"

# prediction = "SchmidtDataset/video_P002_predicted.csv"
# groundtruth = "SchmidtDataset/video_P002_corrected.csv"
# subopt = "SchmidtDataset/video_P002_predicted_subopt.csv"
# videofile = "SchmidtDataset/video_P002.mp4"

prediction = "SchmidtDataset/video_P003_predicted.csv"
groundtruth = "SchmidtDataset/video_P003_corrected.csv"
subopt = "SchmidtDataset/video_P003_predicted_subopt.csv"
videofile = "SchmidtDataset/video_P003.mp4"

# Load as Pandas DataFrame
pred_src = utils.loadDataFrame(prediction)
gt_src = utils.loadDataFrame(groundtruth)
subopt_src = utils.loadDataFrame(subopt)

# Drop duplicates, interpolate, and filter sperm
pred_dd = utils.dropDuplicates(pred_src)
gt_dd = utils.dropDuplicates(gt_src)
subopt_dd = utils.dropDuplicates(subopt_src)

pred = utils.interpolateTracks(pred_dd)
gt = utils.interpolateTracks(gt_dd)
subopt = utils.interpolateTracks(subopt_dd)

pred = metrics.filterSperm(pred)
gt = metrics.filterSperm(gt)
subopt = metrics.filterSperm(subopt)


# print(len(pred_src), len(gt_src), len(subopt_src))
# print("Unique sperm in prediction source:", len(pred_src['sperm'].unique()))
# print("Unique sperm in ground truth source:", len(gt_src['sperm'].unique()))
# print("Unique sperm in suboptimal source:", len(subopt_src['sperm'].unique()))

# print(len(pred), len(gt), len(subopt))
# # Count number of unique sperm in each DataFrame
# print("Unique sperm in prediction:", len(pred['sperm'].unique()))
# print("Unique sperm in ground truth:", len(gt['sperm'].unique()))
# print("Unique sperm in suboptimal:", len(subopt['sperm'].unique()))



In [ ]:
# Compute stats for each file
ps = 1/1.0476  # pixel size in micrometers
fps = 9    # frames per second
gt = stats.computeAllStats(gt,fps=fps,pixel_size=ps,interpolate=False)
pred = stats.computeAllStats(pred,fps=fps,pixel_size=ps,interpolate=False)
subopt = stats.computeAllStats(subopt,fps=fps,pixel_size=ps,interpolate=False)



In [ ]:
# Compute earth movers distance between distributions
import pandas as pd
from scipy.stats import wasserstein_distance

emd_df = pd.DataFrame({"Method": ["Prediction", "Suboptimal"], "VAP": [None, None], "VSL": [None, None], "VCL": [None, None], "ALH_mean": [None, None], "ALH_max": [None, None], "BCF": [None, None]})


# Compute EMD for each statistic
for stat in ["VAP", "VSL", "VCL", "ALH_mean", "ALH_max", "BCF"]:
    emd_df.loc[0, stat] = wasserstein_distance(gt[stat], pred[stat])
    emd_df.loc[1, stat] = wasserstein_distance(gt[stat], subopt[stat])

# Print the EMD DataFrame
print(emd_df)

In [ ]:
#print(pred.head())


# Print maxes of each stat for each file as a dataframe
summary_df = pd.DataFrame({
    'Stat': ['VAP', 'VSL', 'VCL', 'ALH_mean', 'ALH_max', 'BCF'],
    'Prediction': [pred['VAP'].max(), pred['VSL'].max(), pred['VCL'].max(), pred['ALH_mean'].max(), pred['ALH_max'].max(), pred['BCF'].max()],
    'Ground Truth': [gt['VAP'].max(), gt['VSL'].max(), gt['VCL'].max(), gt['ALH_mean'].max(), gt['ALH_max'].max(), gt['BCF'].max()],
    'Suboptimal': [subopt['VAP'].max(), subopt['VSL'].max(), subopt['VCL'].max(), subopt['ALH_mean'].max(), subopt['ALH_max'].max(), subopt['BCF'].max()]
})

print(summary_df)

#print("Mins:",gt["VAP"].min(), gt["VSL"].min(), gt["VCL"].min(), gt["ALH_mean"].min(), gt["ALH_max"].min(), gt["BCF"].min())


In [ ]:
# Side by side box plots

pred_vap = pred.groupby('sperm').first()['VAP']
pred_vsl = pred.groupby('sperm').first()['VSL']
pred_vcl = pred.groupby('sperm').first()['VCL']

gt_vap = gt.groupby('sperm').first()['VAP']
gt_vsl = gt.groupby('sperm').first()['VSL']
gt_vcl = gt.groupby('sperm').first()['VCL']

subopt_vap = subopt.groupby('sperm').first()['VAP']
subopt_vsl = subopt.groupby('sperm').first()['VSL']
subopt_vcl = subopt.groupby('sperm').first()['VCL']


import math
vap_max = math.ceil(max(pred_vap.max(), gt_vap.max(), subopt_vap.max()))
vsl_max = math.ceil(max(pred_vsl.max(), gt_vsl.max(), subopt_vsl.max()))
vcl_max = math.ceil(max(pred_vcl.max(), gt_vcl.max(), subopt_vcl.max()))


# Create dataframe for box plots side-by-side with seaborn
import pandas as pd

# Concatenate the series for each method
df = pd.DataFrame({
    'Data': (['Actual'] * len(gt_vap)) + (['Optimal'] * len(pred_vap)) + (['Suboptimal'] * len(subopt_vap)),
    'VAP': pd.concat([gt_vap, pred_vap, subopt_vap], ignore_index=True),
    'VSL': pd.concat([gt_vsl, pred_vsl, subopt_vsl], ignore_index=True),
    'VCL': pd.concat([gt_vcl, pred_vcl, subopt_vcl], ignore_index=True),

})

fig, axs = plt.subplots(3, 1, figsize=(4, 10))

import seaborn as sns
colors = {'Actual': 'purple', 'Optimal': 'teal', 'Suboptimal': 'orange'}
sns.boxplot(x="Data", y="VAP", hue="Data", data=df, palette=colors, ax=axs[0])
sns.boxplot(x="Data", y="VSL", hue="Data", data=df, palette=colors, ax=axs[1])
sns.boxplot(x="Data", y="VCL", hue="Data", data=df, palette=colors, ax=axs[2])

axs[0].set_ylabel("VAP (µm/s)", fontsize=14)
axs[1].set_ylabel("VSL (µm/s)", fontsize=14)
axs[2].set_ylabel("VCL (µm/s)", fontsize=14)

axs[0].set_xlabel("")
axs[1].set_xlabel("")
axs[2].set_xlabel("Video 3", fontsize=14)

plt.tight_layout()
plt.show()

fig.savefig("boxchart3.jpg", dpi=300)

In [ ]:
# Final Version for Paper

# Build 3x3 subplot for each stat and each method of histograms

pred_vap = pred.groupby('sperm').first()['VAP']
pred_vsl = pred.groupby('sperm').first()['VSL']
pred_vcl = pred.groupby('sperm').first()['VCL']

gt_vap = gt.groupby('sperm').first()['VAP']
gt_vsl = gt.groupby('sperm').first()['VSL']
gt_vcl = gt.groupby('sperm').first()['VCL']

subopt_vap = subopt.groupby('sperm').first()['VAP']
subopt_vsl = subopt.groupby('sperm').first()['VSL']
subopt_vcl = subopt.groupby('sperm').first()['VCL']


# Create a figure with 3 rows and 3 columns
fig, axs = plt.subplots(3, 3, figsize=(10, 8))

import math
vap_max = math.ceil(max(pred_vap.max(), gt_vap.max(), subopt_vap.max()))
vsl_max = math.ceil(max(pred_vsl.max(), gt_vsl.max(), subopt_vsl.max()))
vcl_max = math.ceil(max(pred_vcl.max(), gt_vcl.max(), subopt_vcl.max()))

# Create bins for each histogram for each stat
vap_bins = np.linspace(0, vap_max, 20)
vsl_bins = np.linspace(0, vsl_max, 20)
vcl_bins = np.linspace(0, vcl_max, 20)

# Create horizontal violin plots for each stat and each method
axs[0, 0].hist(gt_vap, bins=vap_bins, color='purple', alpha=1.0, density=True)
axs[0, 0].set_title('Ground Truth VAP')
axs[1, 0].hist(pred_vap, bins=vap_bins, color='purple', alpha=1.0, density=True)
axs[1, 0].set_title('Prediction VAP')
axs[2, 0].hist(subopt_vap, bins=vap_bins, color='purple', alpha=1.0, density=True)
axs[2, 0].set_title('Suboptimal VAP')
axs[0, 1].hist(gt_vsl, bins=vsl_bins, color='purple', alpha=1.0, density=True)
axs[0, 1].set_title('Ground Truth VSL')
axs[1, 1].hist(pred_vsl, bins=vsl_bins, color='purple', alpha=1.0, density=True)
axs[1, 1].set_title('Prediction VSL')
axs[2, 1].hist(subopt_vsl, bins=vsl_bins, color='purple', alpha=1.0, density=True)
axs[2, 1].set_title('Suboptimal VSL')
axs[0, 2].hist(gt_vcl, bins=vcl_bins, color='purple', alpha=1.0, density=True)
axs[0, 2].set_title('Ground Truth VCL')
axs[1, 2].hist(pred_vcl, bins=vcl_bins, color='purple', alpha=1.0, density=True)
axs[1, 2].set_title('Prediction VCL')
axs[2, 2].hist(subopt_vcl, bins=vcl_bins, color='purple', alpha=1.0, density=True)
axs[2, 2].set_title('Suboptimal VCL')

# Set x-limits for each subplot
axs[0, 0].set_xlim(0, vap_max)
axs[1, 0].set_xlim(0, vap_max)
axs[2, 0].set_xlim(0, vap_max)
axs[0, 1].set_xlim(0, vsl_max)
axs[1, 1].set_xlim(0, vsl_max)
axs[2, 1].set_xlim(0, vsl_max)
axs[0, 2].set_xlim(0, vcl_max)
axs[1, 2].set_xlim(0, vcl_max)
axs[2, 2].set_xlim(0, vcl_max)

# Set common axis labels
for ax in axs[:,0]:
    ax.set_ylabel('Probability Density', fontsize=12)
for ax in axs[2,:]:
    ax.set_xlabel('Velocity (µm/s)', fontsize=12)

# Adjust layout
plt.tight_layout()

## Experiment 1

In [ ]:
# Final Version for Paper
# Build 2x3 subplot for each stat and each method of histograms

gt_vap = gt.groupby('sperm').first()['VAP']
gt_vsl = gt.groupby('sperm').first()['VSL']
gt_vcl = gt.groupby('sperm').first()['VCL']

subopt_vap = subopt.groupby('sperm').first()['VAP']
subopt_vsl = subopt.groupby('sperm').first()['VSL']
subopt_vcl = subopt.groupby('sperm').first()['VCL']


# Create a figure with 2 rows and 3 columns
fig, axs = plt.subplots(2, 3, figsize=(10, 6))

import math
vap_max = math.ceil(max(pred_vap.max(), gt_vap.max(), subopt_vap.max()))
vsl_max = math.ceil(max(pred_vsl.max(), gt_vsl.max(), subopt_vsl.max()))
vcl_max = math.ceil(max(pred_vcl.max(), gt_vcl.max(), subopt_vcl.max()))

# Create bins for each histogram for each stat
vap_bins = np.linspace(0, vap_max, 20)
vsl_bins = np.linspace(0, vsl_max, 20)
vcl_bins = np.linspace(0, vcl_max, 20)

# Create horizontal violin plots for each stat and each method
axs[1, 0].hist(gt_vap, bins=vap_bins, color='purple', alpha=1.0, density=True)
axs[1, 0].set_title('Actual VAP')
axs[0, 0].hist(subopt_vap, bins=vap_bins, color='purple', alpha=1.0, density=True)
axs[0, 0].set_title('Predicted VAP')
axs[1, 1].hist(gt_vsl, bins=vsl_bins, color='purple', alpha=1.0, density=True)
axs[1, 1].set_title('Actual VSL')
axs[0, 1].hist(subopt_vsl, bins=vsl_bins, color='purple', alpha=1.0, density=True)
axs[0, 1].set_title('Predicted VSL')
axs[1, 2].hist(gt_vcl, bins=vcl_bins, color='purple', alpha=1.0, density=True)
axs[1, 2].set_title('Actual VCL')
axs[0, 2].hist(subopt_vcl, bins=vcl_bins, color='purple', alpha=1.0, density=True)
axs[0, 2].set_title('Predicted VCL')

# Set x-limits for each subplot
axs[0, 0].set_xlim(0, vap_max)
axs[1, 0].set_xlim(0, vap_max)
axs[0, 1].set_xlim(0, vsl_max)
axs[1, 1].set_xlim(0, vsl_max)
axs[0, 2].set_xlim(0, vcl_max)
axs[1, 2].set_xlim(0, vcl_max)

axs[0, 0].set_ylim(0, 0.06)
axs[1, 0].set_ylim(0, 0.06)
axs[0, 1].set_ylim(0, 0.06)
axs[1, 1].set_ylim(0, 0.06)
axs[0, 2].set_ylim(0, 0.06)
axs[1, 2].set_ylim(0, 0.06)

# Set common axis labels
for ax in axs[:,0]:
    ax.set_ylabel('Probability Density', fontsize=12)
for ax in axs[1,:]:
    ax.set_xlabel('Velocity (µm/s)', fontsize=12)

# Adjust layout
plt.tight_layout()
plt.savefig("experiment1.pdf", dpi=300)